# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [15]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHES = 10
LR = 0.01
DEVICE

device(type='cpu')

In [18]:
class CHNet(nn.Module):
  def __init__(self) -> None:
      super(CHNet, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=100, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=100, out_features=100, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=100, out_features=60, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(60),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=60, out_features=30),
          nn.Dropout(0.2),
          nn.BatchNorm1d(30),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=30, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())
  
  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.predict(out)
    return out[:, 0]

In [19]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.MSELoss()
  best_acc = {'train': None, 'test': None}
  net.train()


In [20]:
california_housing = fetch_california_housing()
# Разделим на тестовые и тренировочные данные
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.25, random_state=13)

In [21]:
# Нормализуем данные и подготовим их для дальнейшего использования в нашем dstaset
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [22]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(DEVICE)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(DEVICE)

test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(DEVICE)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(DEVICE)

In [27]:
train_dataset = CHDataset(train_xt, train_yt)
test_dataset = CHDataset(test_xt, test_yt)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

In [28]:
net = CHNet().to(DEVICE)
optimizer = torch.optim.Adam(net.parameters())
criterion = nn.CrossEntropyLoss()

In [32]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 14364, 18916) exited unexpectedly